# Assignment 2
### Ariel Azria
### Python for Data Science

#### Steps 1 and 2

In [1]:
# Very first thing I'd like to do is import packages I'll need
import numpy as np
import pandas as pd
import random as rd

# Importing file
text = open("walden_clean.txt",encoding='utf-8').read()

# Viewing file: print(text)

# I find the file isn't as clean as I want it to be. I will attempt to remove some other punctuation
replacements = [('—', ''),
                ('_', ''), 
                ('***', ''), 
                ('” ','”'),
                ('—      ', ''),
                ("\n", " "),
               ]

for old, new in replacements:
    text_c = text.replace(old, new)

# Code source: https://realpython.com/replace-string-python/

Next, I will change "." to "PERIOD"

In [2]:
# 1) Converting "." to "PERIOD"
text_np = text_c.replace(".", " PERIOD")

To create pairs of consecutive words, I will convert "text_np" to a list of words.

In [3]:
# 2) Creating lists of consecutive words.
type(text_np) # checking type - this is a string. 

# Creating a list of words. I will create a function that takes a variable and transforms it into a clean string, then a list. 
# Source: https://www.geeksforgeeks.org/python-program-convert-string-list/
def convert_words(string):
    string = string.replace("\n", " ") # in case this needs to be done
    li = list(string.split())
    return li
 
# Using the new function
str1 = text_np
large_list = (convert_words(str1))

# Checking my work: print(large_list)

Now that I have a list, I am ready to create pairs of consecutive words. 

In [4]:
# Next, creating two seperate strings of words and inputting into a dataframe
# Source: https://stackoverflow.com/questions/21303224/iterate-over-all-pairs-of-consecutive-items-in-a-list

empty = [] # creating an empty list

for even, odd in zip(large_list, large_list[1:]):
    empty.append([even, odd])

df = pd.DataFrame(empty, columns=['even', 'odd']) # creating a pandas dataframe that has two columns, each with a word we're interested in. 

print(df)

             even        odd
0               ﻿        the
1             the    project
2         project  gutenberg
3       gutenberg      ebook
4           ebook         of
...           ...        ...
124643         to       hear
124644       hear      about
124645      about        new
124646        new     ebooks
124647     ebooks     PERIOD

[124648 rows x 2 columns]


#### Step 3
Count how often each pair of words occurs in the input text and record it for later retrieval. This is the question in which we generate our transition matrix. 

In [5]:
# 1) Initialize an empty dictionary to store all the counts
trans_matrix = {}

# 2) Populating the dictionary. Iterating over the rows of the df:
for index, row in df.iterrows():
    even, odd = row['even'], row['odd']
    # Check if word1 is already in the dictionary
    if even in trans_matrix:
        # Check if word2 is already a key in the nested dictionary
        if odd in trans_matrix[even]:
            # If yes, increment the count
            trans_matrix[even][odd] += 1
        else:
            # If no, add word2 with a count of 1
            trans_matrix[even][odd] = 1
    else:
        # If word1 is not in the dictionary, add it with a nested dictionary containing word2 and count 1
        trans_matrix[even] = {odd: 1}

# Each "word1" is associated with a nested dictionary that contains "word2" as the key and word2 count as a value. 
# The nested dictionary contains word2 as the key and the count as the value

# Viewing the dictionary for the word 'i' to see results. 
# Commenting this out: trans_matrix['i']

# Note: I used ChatGPT to understand how to set up nested dictionaries in a loop. 

#### Step 4
Normalize the counts and turn them into probabilities.

For each word, find all paired words, add up their frequencies and divide each frequency by total. For example, if "i" is associated with "lived" and "am", and they both of occur once, their total occurrence is 2 and the resulting probabilities are "lived": 0.5, "am":0.5

Mathematically, each row of a transition matrix should sum to 1

In [6]:
# Creating a new variable as to not overwrite my transition matrix
trans_matrix_freq = trans_matrix

for key in trans_matrix:
    total_occurrences = sum(trans_matrix[key].values())  # Obtaining total occurrences of all paired words
    for word in trans_matrix[key]:
        trans_matrix_freq[key][word] = trans_matrix[key][word] / total_occurrences  # Calculating a word's probability
        
print(trans_matrix_freq)

{'\ufeff': {'the': 1.0}, 'the': {'project': 0.004270082732852949, 'duty': 0.0008006405124099279, 'use': 0.0010675206832132373, 'terms': 0.0016012810248198558, 'bean-field': 0.0002668801708033093, 'village': 0.006672004270082733, 'ponds': 0.0008006405124099279, 'pond': 0.016946890846010143, 'following': 0.0012009607686148918, 'bulk': 0.0002668801708033093, 'woods': 0.014544969308780357, 'shore': 0.006672004270082733, 'labor': 0.0005337603416066186, 'notice': 0.00013344008540165466, 'circumstances': 0.0002668801708033093, 'like': 0.0016012810248198558, '_i_': 0.00013344008540165466, 'main': 0.0009340805978115826, 'first': 0.00600480384307446, 'narrowness': 0.00013344008540165466, 'rest': 0.0040032025620496394, 'seams': 0.00013344008540165466, 'coat': 0.0002668801708033093, 'chinese': 0.00040032025620496394, 'inhabitants': 0.0008006405124099279, 'face': 0.0012009607686148918, 'sun': 0.008139845209500934, 'heavens': 0.0013344008540165466, 'twist': 0.00013344008540165466, 'neck': 0.00013344

#### Step 5
Generate text based on this transition matrix. 

I will do this step-by-step, and then aggregate the smaller chunks of code into a large loop at the end of the question. 

In [7]:
# A) Obtaining a first word
tm = trans_matrix_freq # Renaming the transition matrix to make it easier to type

first = rd.choice(list(tm.keys())) # Select random word from dictionary by converting to list first. 
#Source: https://stackoverflow.com/questions/4859292/how-can-i-get-a-random-key-value-pair-from-a-dictionary

print(first)

appendages


In [8]:
# B) Finding the pairs associated with first word and their probabilities
#tm[first] - Commenting this out

# Picking next word based on probabilities
# random.choices() works with lists, so I will have to split my hard-worked dictionary back into lists!
# I'll create one list with the words and the second list with the corresponding frequency values
def word_picker(word):
    words = list(tm[word].keys())  # Get list of paired words
    freqs = list(tm[word].values())  # Get corresponding frequencies
    next_word = rd.choices(words, freqs)[0]  # Choose next word based on frequencies
    return next_word
    
word_picker(first) # Seeing if it works based on the first word picked above. 

'dropped'

In [9]:
# C) Having picked a word, use logic from B to select the next word. Do this in a loop.
# The first word can be obtained using the logic from A
# The second word can be obtained using the logic from B
# We'll need to use append() to add these to a list. 

first_sentence = []
first_sentence.append(first)

for _ in range(1):  # I can choose how many words to generate
    next_word = word_picker(first)  # Use word_picker function defined abvove
    first_sentence.append(next_word)  # Append the chosen word to the sentence
    first_sentence = ' '.join(first_sentence) # Converting this to nice words that look like a sentence

print(first_sentence)

appendages dropped


D) If "PERIOD" is picked, start a new sentence. If the sentence is 10 words long, you can just break it and add a new line (this is just to protect against degenerate cases, which should occur rarely).

I will do this problem and generate the 100 sentences in the same loop.

In [10]:
# Coding variables I will be inserting into the text. 
dot = (".")
l_break = ("\n")

for _ in range(100): # Generating 100 sentences
    first_sentence = [] # list for sentences
    for _ in range(10): # creating another loop for the sentences
        one = word_picker(rd.choice(list(tm.keys()))) # start with the first word
        next_word = word_picker(one) # add the next word
        if next_word == "PERIOD": # if the next word is a PERIOD add "." as next word
            first_sentence.append(dot)
            if len(first_sentence) >= 10: # if the first sentence is 10 words long, line break
                first_sentence.append(l_break)
                break
        else:
            first_sentence.append(next_word) # if the next word is not a period, add the next word
    first_sentence = ' '.join(first_sentence) # add the sentences together
    print(first_sentence) # print the resulting text

the in of the it they when . if immediately
” it animal . our be of the ever well
et even various day a sawed was at of and
some cipher i the in having uddîn all or down
the often silence is if the i so what blue
ten this . elsewhere themselves surveyor a our accept of
like a all for paragraph . were gutenberg more memory
the all to thinks “amok” swam boyhood rolled “trotters” their
or saints the and donations the the the who practical
of cockerels is imagination thither have i in is broad
it like the submit into waving . or nearest a
whom without to right—changes #205] appearance muskrats for great fuel
johnswort indian is pole front by for of which to
famine first sold to called before a but clear picking
durum co all by but and s it all booming
he’s shore poverty perhaps was the his and they my
any among and 9 measure town be plantation has at
. strongest than such kind or . the by went
belonged i for that the with origine their our so
manifestly ripe stopped the founding she as thi